In [1]:
import simuPOP as sim
import random as ran
from random import randint
from random import choices

simuPOP Version 1.1.8.3 : Copyright (c) 2004-2016 Bo Peng
Revision 4553 (Feb 11 2017) for Python 3.6.0 (64bit, 0thread)
Random Number Generator is set to mt19937 with random seed 0xfb32a8ec.
This is the standard short allele version with 256 maximum allelic states.
For more information, please visit http://simupop.sourceforge.net,
or email simupop-list@lists.sourceforge.net (subscription required).


In [2]:
#Global Variables
n_people = 20000
n_loci = 16
n_subpop = 3
n_gen = 10

In [3]:
def randomChooser(pop, subPop):
    males = []
    females = []
    # identify males and females in each social rank
    for rank in range(3):
        males.append([x for x in pop.individuals(subPop) \
            if x.sex() == sim.MALE and x.rank == rank])
        females.append([x for x in pop.individuals(subPop) \
            if x.sex() == sim.FEMALE and x.rank == rank])
    #
    iter = 0
    while True:
        # choose a rank randomly
        iter = iter + 1
        #print(iter)
        if iter > 1:
            #last_randnum1 = randnum1
            #last_randnum2 = randnum2            
            if len(males[rank])>1:
                if randint(0,4)>0: #80%
                    del males[rank][randnum1]
            #print('len_male: ',len(males[rank]))
            if len(females[rank])>1:
                if randint(0,4)>0: #80%
                    del females[rank][randnum2]
            #print('len_female: ',len(females[rank]))
        rank = int(pop.individual(randint(0, pop.subPopSize(subPop) - 1), subPop).rank)
        while len(males[rank])==0 or len(females[rank])==0:
            rank = int(pop.individual(randint(0, pop.subPopSize(subPop) - 1), subPop).rank)
        #print('rank: ',rank)
        randnum1 = randint(0, len(males[rank]) - 1)
        #print('randnum1: ',randnum1)
        #print('male_birthyear: ',int(males[rank][randnum1].birth_year))
        randnum2 = randint(0, len(females[rank]) - 1)
        #print('randnum2: ',randnum2)
        #print('female_birthyear: ',int(females[rank][randnum2].birth_year))
        while males[rank][randnum1].birth_year > (females[rank][randnum2].birth_year + 2)\
        or males[rank][randnum1].birth_year < (females[rank][randnum2].birth_year - 12)\
        or males[rank][randnum1].wealth < females[rank][randnum2].wealth: #modified from 22 to 12 on 20180928
            if randint(0,4)==0: #20%  # moved from the end of while loop to here on 20180928
                break
            #print('REDONE')
            randnum1 = randint(0, len(males[rank]) - 1)
            #print('randnum1: ',randnum1)
            #print('male_birthyear: ',males[rank][randnum1].birth_year)
            randnum2 = randint(0, len(females[rank]) - 1)
            #print('randnum2: ',randnum2)
            #print('female_birthyear: ',females[rank][randnum2].birth_year)
        yield males[rank][randnum1], \
            females[rank][randnum2]

In [4]:
pop = sim.Population([n_people]*n_subpop, loci=[n_loci],\
                     infoFields = ['ind_id', 'father_id', 'mother_id','migrate_to','x',\
                                  'rank', 'birth_year', 'state', 'wealth'], 
                     chromTypes=[sim.CHROMOSOME_Y], ancGen=-1)
pop.setVirtualSplitter(sim.SexSplitter())


In [5]:
#Initial surnames
counts=[]
with open('census_lastnames_freq.txt','r') as f:
    for line in f.readlines():
        counts.append(int(line))
freqs=[]
for i in range(len(counts)):
    freqs.append(counts[i]/sum(counts))
for i in range(1, len(freqs)):
    freqs[i] = freqs[i-1] + freqs[i]
for i in range(n_people*n_subpop):
    r = ran.random()
    for j in range(len(freqs)):
        if r < freqs[j]:
            name = j
            break
    pop.individual(i).x =name
thefile = open('surname_origin.txt', 'w')
for item in pop.indInfo('x'):
    thefile.write("%s\n" % item)
thefile.close()

In [6]:
#Initial states
counts=[]
with open('State_statistics.txt','r') as f:
    for line in f.readlines():
        counts.append(int(line))
freqs=[]
for i in range(len(counts)):
    freqs.append(counts[i]/sum(counts))
for i in range(1, len(freqs)):
    freqs[i] = freqs[i-1] + freqs[i]
cumu_state_freqs=freqs
for i in range(n_people*n_subpop): #
    r = ran.random()
    for j in range(len(freqs)):
        if r < freqs[j]:
            state = j
            break
    pop.individual(i).state =state

In [7]:
#Initial incomes
counts=[]
with open('Income_statistics.txt','r') as f:
    for line in f.readlines():
        counts.append(int(line))
freqs=[]
for i in range(len(counts)):
    freqs.append(counts[i]/sum(counts))
for i in range(1, len(freqs)):
    freqs[i] = freqs[i-1] + freqs[i]
cumu_wealth_freqs=freqs
for i in range(n_people*n_subpop): #
    r = ran.random()
    for j in range(len(freqs)):
        if r < freqs[j]:
            wealth = j
            if wealth <= 15:
                rank = 0
            elif wealth <= 39:
                rank = 1
            else:
                rank = 2            
            break
    pop.individual(i).wealth = wealth #wealth level
    pop.individual(i).rank = rank

In [8]:
counts=[]
with open('Birth_Age_statistics.txt','r') as f:
    for line in f.readlines():
        counts.append(int(line))
freqs=[]
for i in range(len(counts)):
    freqs.append(counts[i]/sum(counts))
birth_age_freqs=freqs

In [9]:
incre_rate=[]
with open('Pop_Increase_stats.txt','r') as f:
    for line in f.readlines():
        incre_rate.append(line)

In [10]:
##def demo(gen, pop):
##    #incre_rate = [.333, .335, .356, .276, .15, .145]
##    return [int(x*(1+incre_rate[gen])) for x in pop.subPopSizes()]
def setBirthYear(birth_year):
    age_range = [x for x in range(15, 35)]
    mother_birth_age = choices(age_range, birth_age_freqs)
    new_birth_year = birth_year[1] + mother_birth_age[0]
    min_father_birth_year = birth_year[0] + 16
    if new_birth_year < min_father_birth_year:
        new_birth_year = min_father_birth_year
    return new_birth_year
def setRankandWealth(rank, wealth):
    add_wealth = randint(-1, 1)
    new_wealth = int((wealth[0] + wealth[1]) / 2 + add_wealth)
    if new_wealth < 0:
        mew_wealth =0
    elif new_wealth >40:
        new_walth = 40
    if new_wealth <= 15:
        new_rank = 0
    elif new_wealth <= 39:
        new_rank = 1
    else:
        new_rank = 2
    return [new_rank, new_wealth]
def setState(state):
    freqs= cumu_state_freqs
    move_draw = randint(1,3) # modified from (1,4) on 20180928
    if move_draw == 1:
        new_state = state[0]
    elif move_draw ==2:
        new_state = state[1]
    else:
        r = ran.random()
        for j in range(len(freqs)):
            if r < freqs[j]:
                new_state = j
                break
    return new_state

In [11]:
pop.evolve(
    initOps=[
        sim.InitSex(sex=[sim.MALE, sim.FEMALE]),
        sim.IdTagger(),
        sim.InitInfo(lambda : randint(1720, 1749), infoFields='birth_year'),
        #sim.InitGenotype(freq=[0]*12 + [0.01, 0.03, 0.17, 0.23, 0.28, 0.18, 0.07, 0.03]),#DYS458
        sim.InitGenotype(freq=[0]*12 + [0.1, 0.36, 0.32, 0.16, 0.06], loci=0),#DYS19
        sim.InitGenotype(freq=[0]*9 + [0.02, 0.16, 0.1, 0.15, 0.2, 0.09, 0.08, 0.07, 0.06, 0.04, 0.02, 0.01], loci=1),#DYS385
        sim.InitGenotype(freq=[0]*11 + [0.28, 0.51, 0.2, 0.01], loci=2),#DYS389I
        sim.InitGenotype(freq=[0]*26 + [0.03, 0.19, 0.34, 0.28, 0.12, 0.04], loci=3),#DYS389II
        sim.InitGenotype(freq=[0]*20 + [0.05, 0.10, 0.30, 0.36, 0.17, 0.02], loci=4),#DYS390
        sim.InitGenotype(freq=[0]*8 + [0.04, 0.63, 0.31, 0.02], loci=5),#DYS391
        sim.InitGenotype(freq=[0]*9 + [0.02, 0.40, 0.07, 0.32, 0.16, 0.02, 0.01], loci=6),#DYS392
        sim.InitGenotype(freq=[0]*10 + [0.01, 0.27, 0.54, 0.15, 0.03], loci=7),#DYS393
        sim.InitGenotype(freq=[0]*12 + [0.01, 0.56, 0.34, 0.09], loci=8),#DYS437
        sim.InitGenotype(freq=[0]*8 + [0.07, 0.46, 0.26, 0.19, 0.02], loci=9),#DYS438
        sim.InitGenotype(freq=[0]*9 + [0.11, 0.35, 0.39, 0.13, 0.02], loci=10),#DYS439
        sim.InitGenotype(freq=[0]*16 + [0.02, 0.15, 0.36, 0.33, 0.12, 0.02], loci=11),#DYS448
        sim.InitGenotype(freq=[0]*12 + [0.02, 0.13, 0.50, 0.24, 0.09, 0.02], loci=12),#DYS456        
        sim.InitGenotype(freq=[0]*12 + [0.01, 0.03, 0.17, 0.23, 0.28, 0.18, 0.07, 0.03], loci=13),#DYS458
        sim.InitGenotype(freq=[0]*18 + [0.05, 0.13, 0.28, 0.16, 0.28, 0.08, 0.02], loci=14),#DYS635
        sim.InitGenotype(freq=[0]*9 + [0.06, 0.37, 0.48, 0.09], loci=15),#GATA H4
        sim.PedigreeTagger(output='>>pedigree.ped', outputLoci=range(n_loci))
    ],
    preOps=[
        #sim.StepwiseMutator(rates=[1e-2]*15,loci=range(15), maxAllele=24),
        sim.StepwiseMutator(rates=[1e-3*2.4],loci=0, maxAllele=20),#DYS19
        sim.StepwiseMutator(rates=[1e-3*2.1],loci=1, maxAllele=28),#DYS385
        sim.StepwiseMutator(rates=[1e-3*1.8],loci=2, maxAllele=17),#DYS389I
        sim.StepwiseMutator(rates=[1e-3*2.8],loci=3, maxAllele=36),#DYS389II
        sim.StepwiseMutator(rates=[1e-3*2.3],loci=4, maxAllele=30),#DYS390
        sim.StepwiseMutator(rates=[1e-3*3.0],loci=5, maxAllele=16),#DYS391
        sim.StepwiseMutator(rates=[1e-3*0.55],loci=6, maxAllele=20),#DYS392
        sim.StepwiseMutator(rates=[1e-3*0.89],loci=7, maxAllele=18),#DYS393
        sim.StepwiseMutator(rates=[1e-3*1.5],loci=8, maxAllele=22),#DYS437
        sim.StepwiseMutator(rates=[1e-3*0.43],loci=9, maxAllele=19),#DYS438
        sim.StepwiseMutator(rates=[1e-3*5.7],loci=10, maxAllele=19),#DYS439
        sim.StepwiseMutator(rates=[1e-3*1.6],loci=11, maxAllele=25),#DYS448
        sim.StepwiseMutator(rates=[1e-3*4.8],loci=12, maxAllele=23),#DYS456
        sim.StepwiseMutator(rates=[1e-3*6.4],loci=13, maxAllele=24),#DYS458
        sim.StepwiseMutator(rates=[1e-3*3.8],loci=14, maxAllele=30),#DYS635
        sim.StepwiseMutator(rates=[1e-3*2.2],loci=15, maxAllele=20),#GATA H4
        sim.Migrator(rate=[
            [0, 0.1, 0.1],
            [0.1, 0, 0.1],
            [0.1, 0.1, 0]],
            mode=sim.BY_PROPORTION,
            step = 5),
    ],
    matingScheme=sim.HomoMating(
        sim.PyParentsChooser(randomChooser),
        sim.OffspringGenerator(ops=[
            sim.MendelianGenoTransmitter(),
            sim.IdTagger(),
            sim.PedigreeTagger(output='>>pedigree.ped', outputLoci=range(n_loci)),
            sim.InheritTagger(mode=sim.PATERNAL, infoFields = 'x'),
            sim.PyTagger(setBirthYear),
            sim.PyTagger(setRankandWealth),
            sim.PyTagger(setState),
            ],
                               numOffspring=(sim.POISSON_DISTRIBUTION, 1.6),
                               
                              ),
        #subPopSize=demo,
    ),
    
    gen = n_gen-1
)

9

In [12]:
#output pedigree
for i in range(3):
    infile = open('pedigree.ped','r')
    outfile = open('ped'+str(3-i)+'.txt', 'w')
    j = 0
    for line in infile.readlines():
        if j >= (n_gen-1-i)*n_people*n_subpop and j < (n_gen-i)*n_people*n_subpop:
            outfile.write(line)
        j += 1
    infile.close()
    outfile.close()

In [13]:
#output info fields
for i in range(3):
    pop.useAncestralGen(i)
    thefile = open('surname'+str(3-i)+'.txt', 'w')
    for item in pop.indInfo('x'):
        thefile.write("%s\n" % item)
    thefile.close()

In [14]:
for i in range(3):
    pop.useAncestralGen(i)
    thefile = open('birth_year'+str(3-i)+'.txt', 'w')
    for item in pop.indInfo('birth_year'):
        thefile.write("%s\n" % int(item))
    thefile.close()

In [15]:
for i in range(3):
    pop.useAncestralGen(i)
    thefile = open('rank'+str(3-i)+'.txt', 'w')
    for item in pop.indInfo('rank'):
        thefile.write("%s\n" % int(item))
    thefile.close()

In [16]:
for i in range(3):
    pop.useAncestralGen(i)
    thefile = open('state'+str(3-i)+'.txt', 'w')
    for item in pop.indInfo('state'):
        thefile.write("%s\n" % int(item))
    thefile.close()

In [17]:
for i in range(3):
    pop.useAncestralGen(i)
    thefile = open('wealth'+str(3-i)+'.txt', 'w')
    for item in pop.indInfo('wealth'):
        thefile.write("%s\n" % int(item))
    thefile.close()